<a href="https://colab.research.google.com/github/tx1103mark/tweet-sentiment/blob/master/TPUs_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TPUs in Colab&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>
In this example, we'll work through training a model to classify images of
flowers on Google's lightning-fast Cloud TPUs. Our model will take as input a photo of a flower and return whether it is a daisy, dandelion, rose, sunflower, or tulip.

We use the Keras framework, new to TPUs in TF 2.1.0. Adapted from [this notebook](https://colab.research.google.com/github/GoogleCloudPlatform/training-data-analyst/blob/master/courses/fast-and-lean-data-science/07_Keras_Flowers_TPU_xception_fine_tuned_best.ipynb) by [Martin Gorner](https://twitter.com/martin_gorner).

#### License

Copyright 2019-2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


---


This is not an official Google product but sample code provided for an educational purpose.


## Enabling and testing the TPU

First, you'll need to enable TPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select TPU from the Hardware Accelerator drop-down

Next, we'll check that we can connect to the TPU:

#Data process

In [ ]:
/**
 * Copyright 2020 Huawei Technologies Co., Ltd
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 * http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

#include "tools/converter/legacy_optimizer/fusion/scale_activation_fusion_pass.h"
#include "src/common/log_adapter.h"
#include "tools/common/graph_util.h"
#include "include/errorcode.h"
#include "schema/inner/model_generated.h"

namespace mindspore {
namespace lite {
#define SCALE_ACT_MATCH_PATH_LEN 2
STATUS ScaleActivationFusionPass::Run(MetaGraphT *graph) { return FusionPass::Run(graph); }

STATUS ScaleActivationFusionPass::DefinePattern() {
  auto scaleOp = std::make_shared<PatternOp>();
  scaleOp->id = SCALE_NAME;
  scaleOp->types = {schema::PrimitiveType_Scale};
  auto actOp = std::make_shared<PatternOp>();
  actOp->id = ACTIVATION_NAME;
  actOp->types = {schema::PrimitiveType_Activation};
  actOp->left = scaleOp;

  std::unique_ptr<FusionPattern> fusionPattern(new (std::nothrow) FusionPattern("ScaleActivationFusion"));
  if (fusionPattern == nullptr) {
    MS_LOG(ERROR) << "new fusionPattern failed";
    return RET_ERROR;
  }
  fusionPattern->AddPatternOp(scaleOp);
  fusionPattern->AddPatternOp(actOp);
  fusionPattern->Finish();

  this->patterns.emplace_back(fusionPattern.release());

  return RET_OK;
}

STATUS ScaleActivationFusionPass::DoFusion(MetaGraphT *graph, const std::string &patternName,
                                  std::unordered_map<std::string, std::shared_ptr<Path>> &matchedPath) {
  MS_ASSERT(graph != nullptr);
  if (matchedPath.size() != SCALE_ACT_MATCH_PATH_LEN) {
    MS_LOG(ERROR) << "Mul-Add-Fusion should have two NodeIndex in matchedPair";
    return RET_PARAM_INVALID;
  }

  auto scalePath = matchedPath[SCALE_NAME];
  auto actPath = matchedPath[ACTIVATION_NAME];
  auto &scaleNode = graph->nodes.at(scalePath->nodeIdx);
  auto &actNode = graph->nodes.at(actPath->nodeIdx);
  MS_ASSERT(scaleNode != nullptr);
  MS_ASSERT(actNode != nullptr);
  auto scale_primitive = scaleNode->primitive->value.AsScale();
  auto scale_act_type = scale_primitive->activationType;
  if(scale_act_type  == ActivationType_NO_ACTIVATION) {
    auto activationType = actNode->primitive->value.AsActivation()->type;
    scale_primitive->activationType = activationType;
    // delete act node
    auto status = IsolateOneWayNode(graph, actNode.get());
    if (status != RET_OK) {
      MS_LOG(ERROR) << "IsolateOneWayNode failed";
      return status;
    }
  }
  return RET_OK;
}
}  // namespace lite
}  // namespace mindspore


In [ ]:
/**
 * Copyright 2020 Huawei Technologies Co., Ltd
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 * http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

#ifndef MINDSPORE_PREDICT_SCALE_ACTIVATION_FUSION_PASS_H
#define MINDSPORE_PREDICT_SCALE_ACTIVATION_FUSION_PASS_H

#include <string>
#include <unordered_map>
#include <memory>
#include <algorithm>
#include <utility>
#include "tools/converter/legacy_optimizer/fusion/fusion_pass.h"
#include "tools/common/graph_util.h"

namespace mindspore {
namespace lite {
constexpr const char *SCALE_NAME = "SCALE";

class ScaleActivationFusionPass : public FusionPass {
 public:
  ScaleActivationFusionPass() = default;

  ~ScaleActivationFusionPass() = default;

  STATUS DefinePattern() override;

  STATUS DoFusion(MetaGraphT *graph, const std::string &patternName,
                  std::unordered_map<std::string, std::shared_ptr<Path>> &matchedPath) override;

  STATUS Run(MetaGraphT *graph) override;
};
}  // namespace lite
}  // namespace mindspore
#endif  // MINDSPORE_PREDICT_SCALE_ACTIVATION_FUSION_PASS_H
